In [1]:
from Bio import SeqIO
import numpy as np
from typing import List
import importlib
import fileDistanceProcessing as fdp
import importlib
importlib.reload(fdp) 

<module 'fileDistanceProcessing' from '/home/marko/Desktop/ip2/ip2_projekat/source/fileDistanceProcessing.py'>

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix

In [3]:
X_train, X_test, y_train, y_test = fdp.train_test_virus()

In [4]:
def get_gram(X1, X2, kernel):
    return np.array([[kernel(X1[i], X2[j]) for j in range(len(X2))] for i in range(len(X1))])

In [5]:
distances = fdp.deserialize_and_decompress_distance_matrix("../data/full_distances_matrix.7z")

# metrika zasnovana samo na 5-adic distanci
# instant racunanje iz matrice
def five_adic_distance_from_matrix(x, y):

    i = int(x[-1]) #indeks je na poslednjem mestu u listi
    j = int(y[-1])

    # matrica je u obliku trougla bez dijagonale originalne matrice
    first = min(i, j)
    second = max(i, j) - (first + 1) # veci indeks smanji za broj instanci u redu matrice do dijagonale 

    return 0 if i == j else distances[first][second] #udaljenost x od y

In [6]:
clf1 = svm.SVC(kernel = "precomputed")
clf1.fit(get_gram(X_train, X_train, five_adic_distance_from_matrix), y_train)

/home/marko/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(kernel='precomputed')

In [7]:
y_pred = clf1.predict(get_gram(X_test, X_train, five_adic_distance_from_matrix))

In [8]:
print(f'Accuracy on Custom Kernel: {accuracy_score(y_test, y_pred)}')

Accuracy on Custom Kernel: 0.7234335551985311


In [9]:
labels = y_test["Virus"].unique()
confusion_matrix(y_test, y_pred, labels=labels)

array([[3152,  588,    0],
       [ 611,    0,    0],
       [   6,    0,    0]])

In [14]:
X_train, X_test, y_train, y_test = fdp.train_test_protein()

In [15]:
clf1.fit(get_gram(X_train, X_train, five_adic_distance_from_matrix), y_train)

/home/marko/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(kernel='precomputed')

In [16]:
y_pred = clf1.predict(get_gram(X_test, X_train, five_adic_distance_from_matrix))

In [17]:
print(f'Accuracy on Custom Kernel: {accuracy_score(y_test, y_pred)}')

Accuracy on Custom Kernel: 0.12370897406472343


In [18]:
confusion_matrix(y_test, y_pred)

array([[   0,    0,    1,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [   0,    0, 1371,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [   0,  839,  539,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,  521],
       [   0,    0,    2,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [   0,    0,   34,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [   0,    0,   73,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [   0,    0,   22,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [   0,    0,   42,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],


In [19]:
X_train, X_test, y_train, y_test = fdp.train_test_sars2()

In [20]:
clf1.fit(get_gram(X_train, X_train, five_adic_distance_from_matrix), y_train)

/home/marko/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(kernel='precomputed')

In [21]:
y_pred = clf1.predict(get_gram(X_test, X_train, five_adic_distance_from_matrix))

In [22]:
print(f'Accuracy on Custom Kernel: {accuracy_score(y_test, y_pred)}')

Accuracy on Custom Kernel: 0.07139037433155081


In [23]:
confusion_matrix(y_test, y_pred)

array([[  1,   0,   0,   0,   0, 232,   0,   0,   0,   0, 262,   0,   0],
       [  2,   0,   0,   0,   0, 120,   0,   0,   0,   0,  71,   0,   0],
       [  5,   0,   0,   0,   0, 273,   0,   0,   0,   0, 217,   0,   0],
       [  0,   0,   0,   0,   0, 264,   0,   0,   0,   0, 231,   0,   0],
       [  0,   0,   0,   0,   0, 119,   0,   0,   0,   0,  88,   0,   0],
       [  2,   0,  20,  12,   0, 231,   0,   0,   0,   0, 230,   0,   0],
       [  0,   0,   0,   0,   0, 251,   0,   0,   0,   0, 244,   0,   0],
       [  0,   0,   0,   0,   0,  35,   0,   0,   0,   0,  17,   0,   0],
       [  0,   0,   0,   0,   0,  87,   0,   0,   0,   0,  48,   0,   0],
       [  0,   0,   0,   0,   0,  14,   0,   0,   0,   0,   8,   0,   0],
       [225,   0,   0,   0,   0, 235,   0,   0,   0,   0,  35,   0,   0],
       [  0,   0,   0,   0,   0,   6,   0,   0,   0,   0,   1,   0,   0],
       [  0,   0,   0,   0,   0,  92,   0,   0,   0,   0,  62,   0,   0]])